In [1]:
from database import *

def ticks_to_kbars(ticks, interval='5Min'):

    kbars = pd.DataFrame()
    
    kbars['open'] = ticks['close'].resample(interval).first()
    kbars['close'] = ticks['close'].resample(interval).last()
    kbars['high'] = ticks['close'].resample(interval).max()
    kbars['low'] = ticks['close'].resample(interval).min()
    kbars['volume'] = ticks['volume'].resample(interval).sum()
    
    kbars.dropna(inplace=True)
    
    return kbars

/opt/anaconda3/lib/python3.8/site-packages/shioaji/__init__.py:20: Warning: Mac OS currently don't provide ca module all function about placing order is not working.
  warnings.warn("Mac OS currently don't provide ca module all function about placing order is not working.", Warning)


In [2]:
import talib

def get_technical_indicator(kbars):
    
#     kbars['rsi'] = talib.RSI(kbars.close, timeperiod=14)
    kbars['cci'] = talib.CCI(kbars.high, kbars.low, kbars.close, timeperiod=14)
    
    macd, macdsignal, macdhist = talib.MACD(kbars.close, fastperiod=5, slowperiod=34, signalperiod=5)
    
    kbars['macd'] = macd
    kbars['macdsignal'] = macdsignal
    kbars['macdhist'] = macdhist
    
    kbars['slowk'], kbars['slowd'] = talib.STOCH(kbars.high,
                        kbars.low,
                        kbars.close,
                        fastk_period=9,
                        slowk_period=3,
                        slowk_matype=0,
                        slowd_period=3,
                        slowd_matype=0)
    
    return kbars

In [3]:
import json

with open('config.json') as f:
  config = json.load(f)

api = sj.Shioaji()
api.login(
    person_id = config['account'],
    passwd= config['password'],
    contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done."))

# api.logout()

connection = sqlite3.connect('data.db')

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done.
<SecurityType.Future: 'FUT'> fetch done.
<SecurityType.Stock: 'STK'> fetch done.
<SecurityType.Option: 'OPT'> fetch done.


In [12]:
code = '1517'

tw_calendar = get_calendar('XTAI')

date = pd.to_datetime('2020-7-20')
prev_trading_date = tw_calendar.previous_close(date).date()

ticks = get_ticks(code, prev_trading_date, connection, api)[0].append(get_ticks(code, date, connection, api)[0])
kbars = ticks_to_kbars(ticks)
kbars = get_technical_indicator(kbars)
# kbars = kbars[pd.to_datetime(prev_trading_date).replace(hour=13, minute=30):]
kbars = kbars[date:]

kbars

,open,close,high,low,volume,cci,macd,macdsignal,macdhist,slowk,slowd
ts,,,,,,,,,,,
2020-07-20 09:00:00,10.85,10.65,10.85,10.65,371,-358.974359,-0.273715,-0.249163,-0.024552,-5.921189e-15,1.105289e-14
2020-07-20 09:05:00,10.65,10.45,10.65,10.40,655,-328.550075,-0.359828,-0.286051,-0.073777,3.333333e+00,1.111111e+00
2020-07-20 09:10:00,10.45,10.15,10.45,10.15,499,-266.115702,-0.489960,-0.354021,-0.135939,3.333333e+00,2.222222e+00
2020-07-20 09:15:00,10.15,10.30,10.35,10.10,341,-177.108434,-0.520996,-0.409679,-0.111317,1.166667e+01,6.111111e+00
2020-07-20 09:20:00,10.40,10.35,10.45,10.30,165,-108.238276,-0.516771,-0.445377,-0.071395,1.875000e+01,1.125000e+01
2020-07-20 09:25:00,10.40,10.35,10.40,10.30,56,-91.528545,-0.504272,-0.465008,-0.039264,2.916667e+01,1.986111e+01
2020-07-20 09:30:00,10.35,10.35,10.40,10.30,96,-76.358105,-0.486811,-0.472276,-0.014535,3.125000e+01,2.638889e+01
2020-07-20 09:35:00,10.35,10.35,10.40,10.35,73,-61.840121,-0.466562,-0.470371,0.003809,3.125000e+01,3.055556e+01
2020-07-20 09:40:00,10.40,10.35,10.40,10.30,25,-57.789614,-0.444948,-0.461897,0.016949,3.194444e+01,3.148148e+01


In [5]:
def day_trading_backtest(code, date, connection, api):
    
    tw_calendar = get_calendar('XTAI')
    prev_trading_date = tw_calendar.previous_close(date).date()
    
    ticks = get_ticks(code, prev_trading_date, connection, api)[0].append(get_ticks(code, date, connection, api)[0])
    kbars = ticks_to_kbars(ticks)
    kbars = get_technical_indicator(kbars)
    kbars = kbars[date:]
            
    entry_price = 0
    entry_time = None
    
    exit_price = 0
    exit_time = None
    
    position = 0
            
    for ts in range(len(kbars)):
        
        current_time = kbars.iloc[ts].name + pd.Timedelta(minutes=5)
        print(' start current_time = {}'.format(current_time))
        current_price = kbars['close'][ts]
        print('current_price = {}'.format(current_price))
#         macdhist = kbars.iloc[ts]['macdhist']
#         print('macdhist = {}'.format(macdhist))
#         cci = kbars.iloc[ts]['cci']
#         print('cci = {}'.format(cci))
#         print('end')
        
        if (
#             kbars.iloc[ts - 3:ts]['rsi'].min() < 30 and
#             kbars.iloc[ts - 1]['macdhist'] < 0 and
            current_time < date.replace(hour=13, minute=20, second=0) and
            kbars.iloc[ts]['cci'] > -100 and
            kbars.iloc[ts]['macdhist'] > 0 and
            position == 0
        ):
            
            position = 1          
            entry_price = current_price
            entry_time = current_time.time()

            print('[{}] buy {} at {}'.format(current_time, code, current_price))
        
        elif (
#             kbars.iloc[ts - 3:ts]['rsi'].max() > 70 and
#             kbars.iloc[ts - 1]['macdhist'] > 0 and
            kbars.iloc[ts]['cci'] > 100 and
            kbars.iloc[ts]['macdhist'] > 0 and
            position != 0
        ):
            exit_price = current_price
            exit_time = current_time.time()
            
            print('[{}] sell {} at {}'.format(current_time, code, current_price))
            
            break
            
        elif (
            current_time >= date.replace(hour=13, minute=20, second=0) and
            position != 0
        ):
            
            exit_price = current_price
            exit_time = current_time.time()
            
            print('[{}] sell {} at {}'.format(current_time, code, current_price))
            
            break
            
    if entry_time and exit_time:
        transaction = pd.DataFrame([[date, 
                                     code, 
                                     entry_time,
                                     entry_price, 
                                     position * 1000,
                                     entry_price * position * 1000,
                                     exit_time,
                                     exit_price,
                                     position * 1000,
                                     exit_price * position * 1000]
                                   ],
                                    columns=[
                                        '成交日期', 
                                        '股票代號',
                                        '買進時間',
                                        '買進價格',
                                        '買進股數',
                                        '買進金額',
                                        '賣出時間',
                                        '賣出價格',
                                        '賣出股數',
                                        '賣出金額'])
        return transaction
    else:
        return pd.DataFrame()

In [6]:
day_trading_backtest('2456', pd.to_datetime('2021/3/2'), connection, api)

 start current_time = 2021-03-02 09:05:00
current_price = 114.0
[2021-03-02 09:05:00] buy 2456 at 114.0
 start current_time = 2021-03-02 09:10:00
current_price = 112.5
[2021-03-02 09:10:00] sell 2456 at 112.5


,成交日期,股票代號,買進時間,買進價格,買進股數,買進金額,賣出時間,賣出價格,賣出股數,賣出金額
0,2021-03-02,2456,09:05:00,114.0,1000,114000.0,09:10:00,112.5,1000,112500.0
